# Initial setup

**GCP Credentials**

From a terminal, run the following command to login to your GCP account.

<div class="alert alert-block alert-info">
Do not run this command from this notebook, use the built-in terminal
</div>


In [ ]:
gcloud auth login --update-adc


**Configuration**

Modify the `GCPPROJECT` and `BUCKETNAME` variables before running the next code block.

You can find your GCP project ID by navigating to https://console.cloud.google.com/welcome

![title](assets/gcpproject.png)

For the `BUCKETNAME` variable you should choose a globally unique name. For more
details about bucket naming, see https://cloud.google.com/storage/docs/buckets


In [ ]:
import os
import glob
import rasterio as rio
from rasterio.windows import Window
import rasterio.plot as rp

# change these 2 variables
GCPPROJECT="solar-climber-412810"
BUCKETNAME="tb-be-bigdata"

# leave the following unchanged
RESULTDIR="results"
SERVICE_NAME="bebigdata"
DOCKER_IMAGE=f"eu.gcr.io/{GCPPROJECT}/be:202401"

os.environ["GCPPROJECT"]=GCPPROJECT
os.environ["BUCKETNAME"]=BUCKETNAME
os.environ["SERVICE_NAME"]=SERVICE_NAME
os.environ["DOCKER_IMAGE"]=DOCKER_IMAGE
os.environ["SAEMAIL"]=f"{SERVICE_NAME}@{GCPPROJECT}.iam.gserviceaccount.com"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=f"{os.environ["HOME"]}/.config/gcloud/application_default_credentials.json"


We will now create the GCP ressources needed by this course:

In [ ]:
%%bash

gcloud config set project $GCPPROJECT
gcloud --project=$GCPPROJECT storage buckets create gs://$BUCKETNAME --default-storage-class=standard --location=europe-west1


gcloud --project=$GCPPROJECT iam service-accounts create $SERVICE_NAME
gcloud --project=$GCPPROJECT services enable containerregistry.googleapis.com
gcloud --project=$GCPPROJECT services enable run.googleapis.com
gcloud --project=$GCPPROJECT services enable pubsub.googleapis.com
gcloud --project=$GCPPROJECT auth configure-docker

gcloud --project=$GCPPROJECT storage buckets add-iam-policy-binding gs://$BUCKETNAME \
    --member=serviceAccount:$SAEMAIL \
    --role=roles/storage.objectAdmin

Inspecter la liste des tuiles S2 disponibles:

In [ ]:
! gsutil ls -l gs://tb-be-bigdata/t31tcj/


et selectionner une tuile pour faire quelques visualisations (choisir une tuile plutot grosse afin
de ne pas tomber sur une tuile completement nuageuse ou partiellement couverte).

In [ ]:
s2_tile='gs://tb-be-bigdata/t31tcj/T31TCJ_20230709T104629_TCI.tif'

with rio.open(s2_tile) as st:
    print(f'size = {st.width}x{st.height} pixels')
    print(f'{len(st.indexes)} bands (channels) of type {st.dtypes[0]}')
    print(f'bounds = {st.bounds}')
    print(f'projection = {st.crs}')
    

La quantité de données à visualiser est trop grosse pour être affichée confortablement en pleine résolution
(i.e. sans avoir à trop attendre et/ou sans consommer trop de temps cpu)

In [ ]:
#affichons la donnée complete depuis une overview (=une version sous-echantillonnée précalculée)
#https://rasterio.readthedocs.io/en/stable/api/rasterio.io.html#rasterio.io.DatasetReader.read
with rio.open(s2_tile) as st:
    ovr = st.read(out_shape=(3,st.height//32,st.width//32))
    rp.show(ovr)

In [ ]:
#affichons un extrait de 200x200px en pleine resolution
# https://rasterio.readthedocs.io/en/stable/api/rasterio.windows.html#rasterio.windows.Window
with rio.open(s2_tile) as st:
    extract = st.read(window=Window(5000,5000,200,200), out_shape=(3,200,200))
    rp.show(extract)



In [ ]:
# affichons un extrait de 200x200px autour du centre de toulouse (longitude=1.4395871 latitude=43.5943521)
# https://pyproj4.github.io/pyproj/stable/examples.html#transformations-from-crs-to-crs
# https://github.com/rasterio/affine
from pyproj import CRS
from pyproj import Transformer
with rio.open(s2_tile) as st:
    transformer = Transformer.from_crs('EPSG:4326', st.crs) #EPSG:4326 is lat/lon
    utm_x, utm_y = transformer.transform(43.5943521,1.4395871)
    utm_to_pix = ~st.transform
    pix_x,pix_y = utm_to_pix*(utm_x,utm_y)
    extract = st.read(window=Window(pix_x-100,pix_y-100,200,200))
    rp.show(extract)

# worker

we will be creating a program that exposes an HTTP endpoint that receives
request with a payload of the form

```json
{
    "window":[x0,y0,width,height],
    "destination":"gs://bucket/path-to-result-tile.tif",
    "datasets":["gs://bucket/prefix/T31TCJ_20230102T104441_TCI.tif",
        "gs://bucket/prefix/T31TCJ_20230103T110349_TCI.tif",
        "....and 214 more..."]
}
```
upon receiving an HTTP request, the program should:

- decode the json payload
- for each dataset, extract the buffer of shape (3,width,height) starting at (x0,y0)
- create a resulting buffer of shape (3,width,height) where each pixel corresponds to
  the rgb triplet of median luminance, after having filtered out samples that are equal
  to 0 (no data, i.e. outide the satellite swath) or 255 (saturated, most likely cloud)
- upload the resulting buffer as a COG file to the requested destination


## implementation

An example implementation can be found in the [worker.py](./worker.py) file.

we will first test this code locally on a single tile.
return to the `dispatcher` code from the notebook to understand how we will be
creating json payloads that can be processed by our worker program.

from a terminal, run a local webserver that exposes our worker code with:

<div class="alert alert-block alert-info">
Do not run this command from this notebook, use the built-in terminal
</div>


In [ ]:
gunicorn --bind :8080 --workers 1 --threads 1 --timeout 0 worker:app --reload


and then run the notebook's dispatch block. check the logs of our worker and
make sure that our resulting tile is now available on our bucket.

# dispatcher

ce code permet de generer les payloads JSON permettant de traiter l'ensemble de l'image, en decoupant à la tuile de
1024x1024

In [ ]:
from google.cloud import storage
from google.cloud import pubsub_v1
import json
import requests
import base64


storage_client = storage.Client(project=GCPPROJECT)
pubsub_client = pubsub_v1.PublisherClient()
topic_name = f'projects/{GCPPROJECT}/topics/{SERVICE_NAME}'
#

s2tiles = list(storage_client.list_blobs("tb-be-bigdata",prefix="t31tcj/"))
s2tiles = [f"gs://tb-be-bigdata/{b.name}" for b in s2tiles if b.name.endswith("TCI.tif")]

tilesize=1024
image_height=10980
image_width=10980
# for testing/debugging, only launch a single tile
image_height=1024
image_width=1024

for x in range(0,image_width,tilesize):
    tile_width = tilesize
    if x+tilesize > image_width:
        tile_width = image_width - x
    for y in range(0,image_height,tilesize):
        tile_height = tilesize
        if y+tilesize > image_height:
            tile_height = image_height - y
        
        payload = {
            "datasets": s2tiles,
            "window": [x,y,tile_width,tile_height],
            "destination": f"gs://{BUCKETNAME}/{RESULTDIR}/tile_{x}_{y}.tif",
            #"destination": f"tile_{x}_{y}.tif",

        }

        # pubsub version
        #future = pubsub_client.publish(topic_name, json.dumps(payload).encode("utf-8"))
        #future.result()
        #continue

        # http version
        http_payload = {
            "message": {
                "data": str(base64.b64encode(json.dumps(payload).encode("utf-8")), "utf-8")
            }
        }
        data=json.dumps(http_payload)
        resp = requests.post(f"http://localhost:8081/median", data=data , headers={"Content-Type":"application/json"})
        print(resp.text)



## docker

https://docs.docker.com/get-started/overview/#the-docker-platform

Once your code is working correctly, build it as a docker image so that it can
be hosted on another platform. 


In [ ]:
! docker build -t $DOCKER_IMAGE .
! docker push $DOCKER_IMAGE

we can also run this code locally directly from the docker image, on the :8081 port:
<div class="alert alert-block alert-info">
Do not run this command from this notebook, use the built-in terminal
</div>

In [ ]:
docker run -t -e PORT=8081 -p 8081:8081 $DOCKER_IMAGE


modify the dispatch code so that your test request is sent to this docker instance.
this time the request will fail. why did it fail?

bonus: modify the previous `docker run` command so that it runs correctly. hint: use
a docker volume to mount your local credentials so they are accessible to the userid
running the code in the docker container.

```bash
docker run -t -e PORT=8081 -p 8081:8081 -v $HOME/.config/gcloud:/root/.config/gcloud $DOCKER_IMAGE
```


## cloud run

Cloud Run is a managed compute platform that lets you run containers directly on top of Google's
scalable infrastructure. You can deploy code written in any programming language on Cloud Run
if you can build a container image from it.

https://cloud.google.com/run/docs/overview/what-is-cloud-run

https://console.cloud.google.com/run


```bash
gcloud --project=$GCPPROJECT run deploy $MYNAME --image $DIMAGE --allow-unauthenticated \
--service-account=$SAEMAIL --region=europe-west1 \
--set-env-vars=CPL_MACHINE_IS_GCE=YES \
--memory 2048M --concurrency=1 --max-instances=50
```
The command will print out on which URL your service is listening.
```bash
# you MUST edit this to replace with the url printed out
export RUNURL=https://bebigdata-xxxx-yyyy-zzzz.a.run.app
```

Adapt the dispatch code to stop sending http requests to localhost, but instead
point them to the $RUNURL/median endpoint on cloud run. Post a single tile to
that endpoint and check the cloud run logs that the tile has been processed
without errors.

This time the code running from the exact same docker image did not fail with permission
errors, why? https://cloud.google.com/run/docs/securing/service-identity

# visualisation du résultat

In [ ]:
#affichage d'une seule tuile
result_tile = f"gs://{BUCKETNAME}/{RESULTDIR}/tile_0_0.tif"
with rio.open(result_tile) as st:
    extract = st.read(out_shape=(3,512,512))
    rp.show(extract)


recontruction d'une image COG complete assemblant chacune des tuiles individuelles

In [ ]:
import subprocess
from google.cloud.storage.blob import Blob
from google.cloud import storage

storage_client = storage.Client(project=GCPPROJECT)
#

result_tiles = list(storage_client.list_blobs(BUCKETNAME,prefix=f"{RESULTDIR}/"))
result_tiles = [f"gs://{BUCKETNAME}/{b.name}" for b in result_tiles if b.name.endswith(".tif")]
f = open("results.lst","w")
for t in result_tiles:
    local_file = t.split("/")[-1]
    blob = Blob.from_string(t, storage_client)
    blob.download_to_filename(local_file)
    f.write(f"{local_file}\n")
f.close()

!gdalbuildvrt -input_file_list results.lst results.vrt

with rio.open("results.vrt") as st:
    profile = st.profile
    profile["driver"]="COG"
    profile["compress"]="JPEG"
    profile["tiled"]=True
    profile["blocksize"]=256
    profile["quality"]=90
    with rio.open("results.tif", "w", **profile) as dst:
        dst.write(st.read())

for t in result_tiles:
    local_file = t.split("/")[-1]
    os.remove(local_file)
os.remove("results.lst")
os.remove("results.vrt")
blob = Blob.from_string(f"gs://{BUCKETNAME}/median.tif", storage_client)
blob.upload_from_filename("results.tif")
os.remove("results.tif")


In [ ]:
median_result = f"gs://{BUCKETNAME}/median.tif"
with rio.open(median_result) as st:
    overview = st.read(out_shape=(3,512,512))
    rp.show(overview)